In [1]:
from copy import deepcopy
import os
import math
import shutil
import re
from glob import glob
from collections import defaultdict

import numpy as np
from tqdm import tqdm
import pandas as pd
pd.set_option('display.max_colwidth', None)
import rispy

In [2]:
dump_dir = '../data/bntl-db-csv-dump-4March2024'

### Recensies

Link reviews to the book that they review.

In [3]:
publications = pd.read_csv(f"{dump_dir}/publications.csv", header=0, parse_dates=['creation_date', 'modification_date'])
publications['id'] = publications['id'].fillna('')
publications

,id,ppn,type,jaar,title_description,abstract,url,containing_publication_id,containing_publication_ppn,page_numbers,creator,creation_date,modifier,modification_date,canonical_url
0,254444,NaN,artikeltijdschrift,2009,"Campenhout, Frans van. Ropsiennes, de nieuwe dichtbundel van Mark Meekers.\r\nIn: Verba (Brussel): 13 (2009) 2 (mei-juli) 14.","Over: Meekers, Mark. Ropsiennes: gedichten bij het leven en werk van Félicien Rops (1833-1898). Leuven: P, 2009.",NaN,NaN,NaN,NaN,19,2009-06-26 11:32:26,19.0,2009-06-26 11:45:26,campenhout_frans_van_ropsienne/campenhout_frans_van
1,254442,NaN,artikeltijdschrift,2009,"Campenhout, Frans van. Dichteres en romancière Liane Bruylants overleden.\r\nIn: Verba (Brussel): 13 (2009) 2 (mei-juli) 5-8.",NaN,NaN,NaN,NaN,NaN,19,2009-06-26 11:27:36,19.0,2009-06-26 11:46:40,campenhout_frans_van_dichteres/campenhout_frans_van
2,313784,NaN,artikeltijdschrift,2018,"Lievois, Katrien, Hanne Kloots & Noureddine Nahed. Kif-kif?: Arabische ontleningen in de Nederlandse, Spaanse en Arabische versie van ""Kiffe kiffe demain"".\r\nIn: Meertaligheid. Speciaal gedeelte van: Filter (Nijmegen): 25 (2018) 3 (sep) 37-47.","P. 46 Noten; p. 46 Bibliografie; p. 47 Bijlage: vertalingen van ""Kiffe kiffe demain"".",NaN,NaN,NaN,NaN,451,2023-11-29 16:36:19,451.0,2023-11-29 16:40:08,lievois_katrien_hanne_kloots_n/lievois_katrien
3,171436,862346606,artikeltijdschrift,1965,"Interprovinciale Prijs voor monografie aan een Westvlaming, [door] A.S.\r\nIn: West-Vlaanderen: 14 (1965) 79 (jan-feb) 66-67.","N.a.v.: Deblaere, Albert. De mystieke schrijfster Maria Petyt, (1623-1677). Gent: Secretarie der Academie, 1962.",NaN,166925.0,853271437,"vol. 14 (1965), afl. 79 (jan-feb), p. 66-67.",1,NaT,19.0,2009-06-26 10:25:49,interprovinciale_prijs_voor_mo
4,246627,NaN,artikelboek,2004,"Willaert, Frank. Margaret's booklets: memory in ""Vanden seven sloten"" by Jan van Ruusbroec.\r\nIn: Medieval memory: image and text; edited by Frank Willaert, Herman Braet, Tom Mertens, Theofiel Venckeleer. Turnhout: Brepols, 2004, p. 99-128.\r\n(Textes et études du Moyen Âge; 27).","Lezing, gehouden op het congres ""Memory in the Middle Ages"" aan de Universiteit Antwerpen, 7-9 maart 2002.",NaN,NaN,NaN,NaN,16,2008-06-18 12:05:19,16.0,2008-06-18 12:15:23,willaert_frank_margaret_s_book/willaert_frank
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302861,254432,NaN,recensie,1962,"Plard, Henri. Une mystique flamande retrouvée: Maria Petyt (1623-1677).\r\nIn: Études germaniques (Paris): 17 (1962) 3 (juill-sep) 347-354.",NaN,NaN,NaN,NaN,NaN,19,2009-06-26 10:31:29,18.0,2014-01-10 12:31:40,plard_henri_une_mystique_flama/plard_henri
302862,129705,NaN,recensie,2003,"Offermans, Cyrille. \r\nIn: Ons erfdeel (Rekkem): 46 (2003) 3 (juni) 446-448.",NaN,NaN,NaN,NaN,NaN,1,NaT,19.0,2012-12-06 10:04:44,cyrille_offermans_in_ons_erfde/offermans_cyrille_albert_willem_3
302863,129707,241847133,boek,2003,"Nederlands-Afghaanse grammatica = Nīdarlandī-Paśtū paśwayah / Sayed Nasir Ahmad (Sayyid Naṣīr Aḥmad). - 3e, herz. dr. - Utrecht : Nederlands Centrum Buitenlanders, cop. 2003. - XIV, 182 p. ; 24 cm Tekst in het Nederlands en het Pasjtoe. - Ook o.d.t.: Nederlands-Pashto grammatica. - Met lit. opg. ISBN 90-5517-169-7.",P. 181-182 Literatuur.\n1e-2e dr. ?,NaN,NaN,NaN,NaN,1,NaT,NaN,NaT,nederlands-afghaanse_grammatic/nasir_ahmad_sayed
302864,254436,NaN,recensie,1964,"Degroote, Gilbert.\r\nIn: Belgisch tijdschrift voor filologie en geschiedenis (Brussel): 42 (1964) 1, 152-153.",NaN,NaN,NaN,NaN,NaN,19,2009-06-26 10:37:20,18.0,2014-01-10 12:33:47,degroote_gilbert_in_revue_belg/degroote_gilbert


In [4]:
publications['title_description'] = [' '.join(p.split()).strip() for p in publications['title_description']]

In [5]:
publications[publications['title_description'] == 'Royen, P. Gerlach O.F.M. In: Levende talen: (1947) 190-191.']

,id,ppn,type,jaar,title_description,abstract,url,containing_publication_id,containing_publication_ppn,page_numbers,creator,creation_date,modifier,modification_date,canonical_url
26,272188,NaN,recensie,1947,"Royen, P. Gerlach O.F.M. In: Levende talen: (1947) 190-191.",NaN,NaN,NaN,NaN,NaN,19,2011-10-06 13:32:35,19.0,2011-10-06 13:33:58,royen_p_gerlach_o_f_m_in_leven/royen_o_f_m_nicolaus_jacobus_hubertus


In [6]:
publications[publications['title_description'] == 'Caron, Willem Johannes Hubertus. Klank en teken bij Erasmus en onze oudste grammatici. Groningen [etc.]: Wolters, 1947. XII, 140 p. + stellingen.']

,id,ppn,type,jaar,title_description,abstract,url,containing_publication_id,containing_publication_ppn,page_numbers,creator,creation_date,modifier,modification_date,canonical_url
255901,273,036356891,boek,1947,"Caron, Willem Johannes Hubertus. Klank en teken bij Erasmus en onze oudste grammatici. Groningen [etc.]: Wolters, 1947. XII, 140 p. + stellingen.","Proefschrift Vrije Universiteit Amsterdam.\r\nOok in: Caron, W. J. H. Klank en teken: verzamelde taalkundige studies; [voor herdr. gereedgemaakt en van een bibliografie voorz. door J. Knol; met medew. van H. J. Brinkman... et al.]. Groningen: Wolters-Noordhoff, 1972, p. 1-128.\r\nP. XII Lijst van gebruikte afkortingen; p. 141 Personenregister.\r\nO.a. ook over: Richard Dafforne, Anthonis de Huybert, Lambert ten Kate, Petrus Montanus, Arnold Moonen, H. L. Spiegel en Wigardus à Winschooten.",NaN,NaN,NaN,NaN,1,NaT,19.0,2011-10-06 13:37:17,klank_en_teken_bij_erasmus_en/caron_willem_johannes_hubertus


In [7]:
publications[publications['title_description'].str.contains('Ene andre tale:')]

,id,ppn,type,jaar,title_description,abstract,url,containing_publication_id,containing_publication_ppn,page_numbers,creator,creation_date,modifier,modification_date,canonical_url
192626,279543,NaN,boek,2012,"Ene andre tale: tendensen in de Middelnederlandse late ridderepiek; onder redactie van An Faems en Marjolein Hogenbirk. Hilversum: Verloren, 2012. 318 p. (Middeleeuwse studies en bronnen; 131).","ISBN 978-90-8704-224-0.\r\n\r\nP. 11-36 De Middelnederlandse late ridderepiek: 'bleeke spookgestalten' krijgen kleur; [inleiding], door An Faems; p. 291-311 Literatuurlijst; p. 312-316 Register van auteurs en werken; p. 317-318 Register van handschriften en drukken.",NaN,NaN,NaN,NaN,17,2012-12-18 10:33:38,17.0,2012-12-18 10:47:42,ene_andre_tale_tendensen_in_de/faems_an
192627,279544,NaN,artikelboek,2012,"Janssens, Jef. De Middelnederlandse ridderepiek in de veertiende eeuw: postmodern of voorzichtig vernieuwend? In: Ene andre tale: tendensen in de Middelnederlandse late ridderepiek; onder redactie van An Faems en Marjolein Hogenbirk. Hilversum: Verloren, 2012, p. 37-52. (Middeleeuwse studies en bronnen; 131).",NaN,NaN,NaN,NaN,NaN,17,2012-12-18 10:49:56,17.0,2012-12-18 11:04:27,janssens_jef_de_middelnederlan/janssens_jozef_d
192636,279545,NaN,artikelboek,2012,"Meulen, Janet F. van der. Vrouwen van Avesnes: een nieuwe Alexander in de Lage Landen. In: Ene andre tale: tendensen in de Middelnederlandse late ridderepiek; onder redactie van An Faems en Marjolein Hogenbirk. Hilversum: Verloren, 2012, p. 55-81. (Middeleeuwse studies en bronnen; 131).",NaN,NaN,NaN,NaN,NaN,17,2012-12-18 11:04:28,17.0,2012-12-18 11:30:35,meulen_janet_f_van_der_vrouwen/meulen_janet_f_van_der
192646,279549,NaN,artikelboek,2012,"Reynders, Anne. ""Ghi heren, ic houde in ware wort dat ghi van Alexandre gehort hebt"": de Middelnederlandse vertalingen van de Oudfranse ""Florimont"" en ""Voeux du paon"". In: Ene andre tale: tendensen in de Middelnederlandse late ridderepiek; onder redactie van An Faems en Marjolein Hogenbirk. Hilversum: Verloren, 2012, p. 83-101. (Middeleeuwse studies en bronnen; 131).",NaN,NaN,NaN,NaN,NaN,17,2012-12-18 11:30:35,18.0,2014-02-18 08:58:59,reynders_anne_ghi_heren_ic_hou/reynders_anne
192647,279552,NaN,artikelboek,2012,"Brandsma, Frank. Florimont 2.0 (editie, vertaling, context); [kritische kanttekeningen en redactie: Bart Besamusca]. In: Ene andre tale: tendensen in de Middelnederlandse late ridderepiek; onder redactie van An Faems en Marjolein Hogenbirk. Hilversum: Verloren, 2012, p. 103-121. (Middeleeuwse studies en bronnen; 131).",NaN,NaN,NaN,NaN,NaN,17,2012-12-18 11:42:01,18.0,2014-02-18 09:00:05,brandsma_frank_florimont_2_0_e/brandsma_frank_p_c
192655,279558,NaN,artikelboek,2012,"Lens, Mieke. Huge van Bordeeus: avontuur voor de stedelijke burgerij? In: Ene andre tale: tendensen in de Middelnederlandse late ridderepiek; onder redactie van An Faems en Marjolein Hogenbirk. Hilversum: Verloren, 2012, p. 125-150. (Middeleeuwse studies en bronnen; 131).",NaN,NaN,NaN,NaN,NaN,17,2012-12-18 11:52:14,17.0,2012-12-18 12:18:43,lens_mieke_huge_van_bordeeus_a/lens_maria_johanna
192668,279570,NaN,artikelboek,2012,"Claassens, Geert H.M. ""Doe leefde hi soe heilichlike"": Seghelijn van Jherusalem tussen ridderepiek en hagiografie. In: Ene andre tale: tendensen in de Middelnederlandse late ridderepiek; onder redactie van An Faems en Marjolein Hogenbirk. Hilversum: Verloren, 2012, p. 195-212. (Middeleeuwse studies en bronnen; 131).",NaN,NaN,NaN,NaN,NaN,17,2012-12-18 13:48:00,31.0,2020-05-29 13:52:35,claassens_geert_h_m_doe_leefde/claassens_gerard_henricus_marie
192673,279569,NaN,artikelboek,2012,"Tersteeg, Jacques. Een Vlaamse schandaalgeschiedenis en sleutelroman?: de Middelnederlandse fragmenten van de 'Borchgrave van Couchi' en de oudfranse prozaroman 'Baudoin de Flandre'. In: Ene andre tale: tendensen in de Middelnederlandse late ridderepiek; onder redactie van An Faems en Marjolein Hogenbirk. Hilversum: Verloren, 2012, p. 151-194. (Middeleeuwse s

In [8]:
title2id = dict(zip(publications['title_description'], publications['id']))
title2ptype = dict(zip(publications['title_description'], publications['type']))
id2title = dict(zip(publications['id'], publications['title_description']))

In [9]:
print(title2id['Royen, P. Gerlach O.F.M. In: Levende talen: (1947) 190-191.'])
print(title2id['Caron, Willem Johannes Hubertus. Klank en teken bij Erasmus en onze oudste grammatici. Groningen [etc.]: Wolters, 1947. XII, 140 p. + stellingen.'])

272188
273


In [10]:
print(id2title[272188])
print(id2title[273])

Royen, P. Gerlach O.F.M. In: Levende talen: (1947) 190-191.
Caron, Willem Johannes Hubertus. Klank en teken bij Erasmus en onze oudste grammatici. Groningen [etc.]: Wolters, 1947. XII, 140 p. + stellingen.


In [11]:
references = pd.read_csv(f'{dump_dir}/publication_publications.csv')
rev2ref = dict(zip(references['publication_id'], references['referenced_publication_id']))

In [12]:
print(rev2ref[272188])

273


In [13]:
llm_path = '../data/llm-dump'

for decade_folder in sorted(glob(f'{llm_path}/*')):
    print(':::', decade_folder, ':::')
    decade = decade_folder.split('/')[-1]

    for ris_path in sorted(glob(f'{decade_folder}/*_consolidated.ris')):
        print(ris_path)
        with open(ris_path) as f:
            entries = rispy.load(f, encoding='utf-8')
        for idx, entry in tqdm(list(enumerate(entries))):
            if not 'keywords' in entry or not 'recensie' in entry['keywords']:
                continue
            if 'notes_abstract' in entry:
                pub_title = ' '.join(entry['notes_abstract'].split()).strip()
            else:
                # this means that we weren't able to structure the item via the LLM...
                pub_title = ' '.join(entry['title'].split()).strip()
            
            ref_title = None
            try:
                #print('==============================')
                #print(pub_title)
                pub_id = title2id[pub_title]
                #print(pub_id)
                ref_id = rev2ref[pub_id]
                #print(ref_id)
                ref_title = id2title[ref_id]
                #print(ref_id, ref_title)
            except KeyError:
                pass
            if ref_title:
                entry['reviewed_item'] = ref_title
                entries[idx] = entry

        out_ris_path = ris_path.replace('.ris', f'_link_{decade}.ris')
        with open(out_ris_path, 'w') as bibliography_file:
            rispy.dump(entries, bibliography_file)

::: ../data/llm-dump/1940s :::
../data/llm-dump/1940s/JFULL_consolidated.ris


100%|██████████| 68/68 [00:00<00:00, 590502.43it/s]


::: ../data/llm-dump/1950s :::
../data/llm-dump/1950s/JFULL_consolidated.ris


100%|██████████| 17/17 [00:00<00:00, 631001.49it/s]


::: ../data/llm-dump/1960s :::
../data/llm-dump/1960s/JFULL_consolidated.ris


...

100%|██████████| 442/442 [00:00<00:00, 4101509.66it/s]


::: ../data/llm-dump/2020s :::
../data/llm-dump/2020s/JFULL_consolidated.ris


100%|██████████| 161/161 [00:00<00:00, 1026265.87it/s]

::: ../data/llm-dump/misc :::
../data/llm-dump/misc/JFULL_consolidated.ris



100%|██████████| 6051/6051 [00:00<00:00, 5736829.45it/s]


## Boeken

In [14]:
from collections import defaultdict
ref2revs = defaultdict(list)
for rev, ref in rev2ref.items():
    ref2revs[ref].append(rev)

In [15]:
def get_reviews(pub_title):
    try:
        pub_id = title2id[pub_title]
    except KeyError:
        return None
    
    reviews = []
    for rev in ref2revs[pub_id]:
        try:
            reviews.append(id2title[rev])
        except KeyError:
            continue
    
    if reviews:
        return 'Recensies: ' + ' | '.join(reviews)

In [16]:
print(get_reviews('Horst, J. M. van der. Geschiedenis van de Nederlandse syntaxis. Leuven: Universitaire Pers Leuven, 2008. 2 dl. (2014 p.).'))

Recensies: Debrabandere, Frans. In: Biekorf (Brugge): 109 (2009) 3 (sep) 358-359. | Hüning, Matthias. Kroniek van de taalkunde 2008. In: Internationale Neerlandistiek (Woubrugge): 47 (2009) 3 (okt) 50-54. | Zeijlstra, Hedde. In: Tijdschrift voor Nederlandse taal- & letterkunde, TNTL (Leiden): 126 (2010) 1, 94-95. | Sels, Koen. In: De leeswolf (Antwerpen): 15 (2009) 5 (juni) 377. | Marynissen, Ann. In: Internationale Neerlandistiek (Amsterdam): 48 (2010) 1 (feb) 57-60. | Boonen, Ute K. In: Jubileumnummer 1963-2012 Internationale Neerlandistiek; [redactie Ralf Grüttemeier et al.]. Speciaal nummer van: Internationale neerlandistiek (Amsterdam): 50 (2012) 1 (jan) 149-151. | Devos, Filip. Monumentale historische syntax. In: Over taal (Kortrijk-Heule): 48 (2009) 5 (nov-dec) 136-137.


In [17]:
llm_path = '../data/llm-dump'

for decade_folder in sorted(glob(f'{llm_path}/*')):
    print(':::', decade_folder, ':::')
    decade = decade_folder.split('/')[-1]

    for ris_path in sorted(glob(f'{decade_folder}/*_consolidated.ris')):
        print(ris_path)
        with open(ris_path) as f:
            entries = rispy.load(f, encoding='utf-8')
        
        for idx, entry in tqdm(list(enumerate(entries))):
            if not entry['type_of_reference'] == 'BOOK':
                continue
            if 'notes_abstract' in entry:
                pub_title = ' '.join(entry['notes_abstract'].split()).strip()
            else:
                # this means that we weren't able to structure the item via the LLM...
                pub_title = ' '.join(entry['title'].split()).strip()
            
            reviews = get_reviews(pub_title)
            if reviews:
                entry['research_notes'] = reviews
                entries[idx] = entry

        out_ris_path = ris_path.replace('.ris', f'_link_{decade}.ris')
        with open(out_ris_path, 'w') as bibliography_file:
            rispy.dump(entries, bibliography_file)

::: ../data/llm-dump/1940s :::
../data/llm-dump/1940s/JFULL_consolidated.ris


100%|██████████| 68/68 [00:00<00:00, 1707860.31it/s]


::: ../data/llm-dump/1950s :::
../data/llm-dump/1950s/JFULL_consolidated.ris


100%|██████████| 17/17 [00:00<00:00, 604264.14it/s]


::: ../data/llm-dump/1960s :::
../data/llm-dump/1960s/JFULL_consolidated.ris


...

100%|██████████| 442/442 [00:00<00:00, 5135408.22it/s]


::: ../data/llm-dump/2020s :::
../data/llm-dump/2020s/JFULL_consolidated.ris


100%|██████████| 161/161 [00:00<00:00, 1150396.84it/s]


::: ../data/llm-dump/misc :::
../data/llm-dump/misc/JFULL_consolidated.ris


100%|██████████| 6051/6051 [00:00<00:00, 7119139.83it/s]


## Chapters

In [18]:
llm_path = '../data/llm-dump'

for decade_folder in sorted(glob(f'{llm_path}/*'))[::-1]:
    print(':::', decade_folder, ':::')
    decade = decade_folder.split('/')[-1]

    for ris_path in sorted(glob(f'{decade_folder}/*_consolidated.ris')):
        print(ris_path)
        with open(ris_path) as f:
            entries = rispy.load(f, encoding='utf-8')
        
        for idx, entry in tqdm(list(enumerate(entries))):
            if not entry['type_of_reference'] == 'CHAP':
                continue
            if 'notes_abstract' in entry:
                pub_title = ' '.join(entry['notes_abstract'].split()).strip()
            else:
                # this means that we weren't able to structure the item via the LLM...
                pub_title = ' '.join(entry['title'].split()).strip()
            pub_id = title2id[pub_title]
            containing_title = None
            try:
                containing_id = rev2ref[pub_id]
                containing_title = id2title[containing_id]
            except KeyError:
                continue

            if containing_title:
                print(pub_title[:80], title2id[pub_title])
                print('       ->', containing_title)
                entry['custom3'] = containing_title
                entries[idx] = entry

        out_ris_path = ris_path.replace('.ris', f'_link_{decade}.ris')
        with open(out_ris_path, 'w') as bibliography_file:
            rispy.dump(entries, bibliography_file)



::: ../data/llm-dump/misc :::
../data/llm-dump/misc/JFULL_consolidated.ris


100%|██████████| 6051/6051 [00:00<00:00, 8609136.20it/s]


::: ../data/llm-dump/2020s :::
../data/llm-dump/2020s/JFULL_consolidated.ris


100%|██████████| 161/161 [00:00<00:00, 3517098.67it/s]


::: ../data/llm-dump/2010s :::
../data/llm-dump/2010s/JFULL_consolidated.ris


...

100%|██████████| 190/190 [00:00<00:00, 2296592.97it/s]


::: ../data/llm-dump/1950s :::
../data/llm-dump/1950s/JFULL_consolidated.ris


100%|██████████| 17/17 [00:00<00:00, 848847.24it/s]


::: ../data/llm-dump/1940s :::
../data/llm-dump/1940s/JFULL_consolidated.ris


100%|██████████| 68/68 [00:00<00:00, 2437715.15it/s]


In [19]:
pub_title = 'Caers, Bram en Mike Kestemont. The cultural and historical context of the low countries. In: The Arthur of the Low Countries: the Arthurian legend in Dutch and Flemish literature; Bart Besamusca & Frank Brandsma (ed.). Cardiff: University of Wales Press, 2021, p. 7-30. (Arthurian literature in the Middle Ages; X).'
pub_id = title2id[pub_title]
print(pub_id)

312401


In [20]:
ref2revs[pub_id]

[]

## Themanummers

In [21]:
def special_issue_references(pub_title):
    reviews, articles = [], []

    try:
        pub_id = title2id[pub_title]
    except KeyError:
        return None, None
    
    for entry in ref2revs[pub_id]:
        entry_title = None
        try:
            entry_title = id2title[entry]
        except KeyError:
            continue
        
        if title2ptype[entry_title] == 'recensie':
            reviews.append(entry_title)
        elif title2ptype[entry_title] == 'artikeltijdschrift':
            articles.append(entry_title)
    
    if reviews:
        reviews = 'Recensies: ' + ' | '.join(reviews)
    if articles:
        articles = 'Artikels: ' + ' | '.join(articles)

    return reviews, articles

    
pub_title = 'Theorie en de digitale geesteswetenschappen; red. Mike Kestemont en Dirk van Hulle. Hilversum: Verloren, 2019. 177 p. Speciaal nummer van: Tijdschrift voor Nederlandse taal- en letterkunde, TNTL (Leiden): 135 (2019) 4, 301-477.'
special_issue_references(pub_title)

('Recensies: Deijl, Lucas van der. In: Nederlandse letterkunde (Amsterdam): 25 (2020) 3 (dec) 281-283.',
 "Artikels: Kestemont, Mike & Luc Herman. Kunnen machines (literatuur) lezen? In: Theorie en de digitale geesteswetenschappen; red. Mike Kestemont en Dirk van Hulle. Speciaal nummer van: Tijdschrift voor Nederlandse taal- en letterkunde, TNTL (Leiden): 135 (2019) 4, 316-328. | Velde, Freek van de, Karlien Franco & Dirk Geeraerts. Reality check voor de kwantitatieve Nederlandse taalkunde: laveren tussen de Scylla van het conservatisme en de Charybdis van de zelfgenoegzaamheid. In: Theorie en de digitale geesteswetenschappen; red. Mike Kestemont en Dirk van Hulle. Speciaal nummer van: Tijdschrift voor Nederlandse taal- en letterkunde, TNTL (Leiden): 135 (2019) 4, 329-343. | Zundert, J. J. van & K. H. van Dalen-Oskam. Joris en Karina's Holistisch Letterkundig Onderzoeksbureau. In: Theorie en de digitale geesteswetenschappen; red. Mike Kestemont en Dirk van Hulle. Speciaal nummer van: T

In [22]:
llm_path = '../data/llm-dump'

for decade_folder in sorted(glob(f'{llm_path}/*'))[::-1]:
    print(':::', decade_folder, ':::')
    decade = decade_folder.split('/')[-1]

    for ris_path in sorted(glob(f'{decade_folder}/*_consolidated.ris')):
        print(ris_path)
        with open(ris_path) as f:
            entries = rispy.load(f, encoding='utf-8')
        
        for idx, entry in tqdm(list(enumerate(entries))):
            if not entry['type_of_reference'] == 'JFULL':
                continue
            if 'notes_abstract' in entry:
                pub_title = ' '.join(entry['notes_abstract'].split()).strip()
            else:
                # this means that we weren't able to structure the item via the LLM...
                pub_title = ' '.join(entry['title'].split()).strip()
            
            reviews, articles = special_issue_references(pub_title)

            if reviews:
                entry['research_notes'] = reviews
            if articles:
                entry['custom4'] = articles
            
            entries[idx] = entry

        out_ris_path = ris_path.replace('.ris', f'_link_{decade}.ris')
        with open(out_ris_path, 'w') as bibliography_file:
            rispy.dump(entries, bibliography_file)

::: ../data/llm-dump/misc :::
../data/llm-dump/misc/JFULL_consolidated.ris


100%|██████████| 6051/6051 [00:00<00:00, 196966.57it/s]


::: ../data/llm-dump/2020s :::
../data/llm-dump/2020s/JFULL_consolidated.ris


100%|██████████| 161/161 [00:00<00:00, 136475.94it/s]

::: ../data/llm-dump/2010s :::
../data/llm-dump/2010s/JFULL_consolidated.ris


...

100%|██████████| 190/190 [00:00<00:00, 332742.28it/s]


::: ../data/llm-dump/1950s :::
../data/llm-dump/1950s/JFULL_consolidated.ris


100%|██████████| 17/17 [00:00<00:00, 97809.56it/s]


::: ../data/llm-dump/1940s :::
../data/llm-dump/1940s/JFULL_consolidated.ris


100%|██████████| 68/68 [00:00<00:00, 139332.03it/s]
